# code结构
    + 读取txt,建立一个mappoint类
    + 读取整个rgb文件夹，用一个列表维护所有的图像文件名,列表mats
    + 读取整个txt，对里面的结果进行解析，并不断的新建mappoint,所有的mappoint用一个points列表维护
        + mappoint里面的信息，用字典维护，键是matid，值是x坐标,y坐标
        + class_id,class_name,置信度，要不要放在字典里呢
    + 用一个字典维护，mats 和　points的对应关系，一个mat_id可以对应多个point_id,一个point_id也可以对应多个mat_id,还有必要吗，有了从mpid->matid的查询，从mat_id查找所有的mpid,似乎不是很重要了


+ 遍历每一个mappoint
    ++ 如果mappoint的某一帧结果已经被计算了，那么就用那一帧的结果分配类别
    ++ 在字典里找出，所有相同类别的点，
    ++ 对每个点计算和当前点的共视关系，如果存在共视，那么找到这个共视下的结果，属于同一实例，break，不属于，继续循环
    ++ 如果一直没找到实例，那就分配一个新的实例
    ++ 这个方法感觉相对轻量级
    
这里有个假设，分类网络对不同帧的分类结果一样，没有考虑各种冲突什么的，
还有就是orbslam保存的共视帧，不知道会不会少o,

In [19]:
class MapPoint():
    def __init__(self, global_id):
        self.global_id = global_id
        self.mats = {}
        self.class_id = -1
        self.class_name = 'background'
        self.score = 0
        self.instance_id = ''

In [20]:
import os

path = 'rgb/'
dirs = os.listdir(path)
#所有图片文件名
mats = sorted(dirs, key = lambda x: float(x[:-4]))
#len(mats) = 613,600张图片
#dirs = sorted([float(x[:-4]) for x in dirs])
#mats = [str(x)+'.png' for x in dirs]


In [21]:
cnt = 0
points = []
mat_set = []
with open("mpoint_saving.txt", "r") as f:
    for line in f: # for line in f 竟然就是逐行读去了，醉了醉了
        s = line.strip('\n').split(' ')
        #print(s)
        if s[0] == 'global':
            point = MapPoint(int(s[3]))
            points.append(point)
            point.mats[int(s[4])] = [float(s[6]), float(s[7])]
            cnt += 1
            mat_set.append(int(s[4]))
            #point.mat_id = int(s[4])
            #point.x_corrdi = float(s[6])
            #point.y_corrdi = float(s[7])
        elif s[0] == "mappoint":
            pass
        else:
            point.mats[int(s[0])] = [float(s[2]), float(s[3])]
            mat_set.append(int(s[0]))
        #if cnt > 10:
        #    break
mat_set = set(mat_set)
print("{} key frames and {} mppoints are envolved".format(len(mat_set), cnt))
#600张图片里面的１０１张被选作为关键帧，

101 key frames and 3205 mppoints are envolved


In [22]:
import detectron2
import cv2
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import MetadataCatalog

cfg = get_cfg()
cfg.merge_from_file("/home/zherlock/InstanceDetection/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")   # load values from a file
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
cfg.merge_from_list(["MODEL.WEIGHTS", "/home/zherlock/InstanceDetection/detectron2/pre_train_model/model_final_f10217.pkl","MODEL.DEVICE","cpu"]) 
#print(cfg.dump())
predictor = DefaultPredictor(cfg)

metadata = MetadataCatalog.get(cfg.DATASETS.TRAIN[0])
classnames = metadata.get("thing_classes", None) #所有label名,80类物体哇

In [ ]:
#挺多地图点都出现在同一个关键帧里，用一个hashmap存储一个图片的结果，如果遇到相同的就查找就好
#并不是每个点都属于物体呢,挺多关键点在背景里面的

In [ ]:
#图片是640*480的，可能是行列弄反了吧
#len(output["instances"].pred_masks[0] is 480
#len(output["instances"].pred_masks[0][0] is 640

In [23]:
def getoutput(mid:int):
    if mid in res_output:
        return res_output[mid]
    mat_path = os.path.join(mat_dir, mats[mid])
    im = cv2.imread(mat_path)
    output = predictor(im)
    res_output[mid] = output
    return output

In [24]:
def convisual(pa:MapPoint, pb:MapPoint):
    if not pb:
        return None
    set_a = set([mid for mid in pa.mats])
    set_b = set([mid for mid in pb.mats])
    jiaoji = set_a & set_b
    if jiaoji:
        return jiaoji
    else:
        return None

In [25]:
def assign_inst(p:MapPoint):
    candidates = []
    if p.class_name in label_dic:
        candidates = label_dic[p.class_name]
    for candy in candidates:
        mid_set = convisual(p, candy)
        mid_set = mid_set if mid_set else []
        for mid in mid_set:
            output = getoutput(mid)
            for i in range(len(output["instances"])):
                #if output["instances"].pred_classes[i] == p.class_id:
                    if output["instances"].pred_masks[i][int(p.mats[mid][1])][int(p.mats[mid][0])]:
                        if output["instances"].pred_masks[i][int(candy.mats[mid][1])][int(candy.mats[mid][0])]:
                            #print('now link {} and {} in mat {}'.format(p.global_id, candy.global_id,mid))
                            #print('this is instance {}'.format(i))
                            p.instance_id = candy.instance_id
                            label_dic[p.class_name].append(p)
                            print('point {} belongs to instance {}'.format(p.global_id, p.instance_id))
                            return
    #新建一个instance
    if p.class_name in instance_dic:
        i = instance_dic[p.class_name][-1]
        instance_dic[p.class_name].append(i+1)
        name = str(p.class_name) + ' ' + str(instance_dic[p.class_name][-1])
    else:
        instance_dic[p.class_name] = [1]
        name = str(p.class_name) + ' ' + str(int(1))
    p.instance_id = name
    print('point {} belongs to instance {}'.format(p.global_id, p.instance_id))
    if p.class_name in label_dic:
        label_dic[p.class_name].append(p)
    else:
        label_dic[p.class_name] = [p]
    return

In [ ]:
'''
def assign_inst(p:MapPoint):
    candidates = []
    if p.class_name in label_dic:
        candidates = label_dic[p.class_name]
    for candy in candidates:
        mid = convisual(p, candy)
        if mid:
            output = getoutput(mid)
            for i in range(len(output["instances"])):
                #if output["instances"].pred_classes[i] == p.class_id:
                    if output["instances"].pred_masks[i][int(p.mats[mid][1])][int(p.mats[mid][0])]:
                        if output["instances"].pred_masks[i][int(candy.mats[mid][1])][int(candy.mats[mid][0])]:
                            p.instance_id = candy.instance_id
                            label_dic[p.class_name].append(p)
                            print('point {} belongs to instance {}'.format(p.global_id, p.instance_id))
                            return
    #新建一个instance
    if p.class_name in instance_dic:
        i = instance_dic[p.class_name][-1]
        instance_dic[p.class_name].append(i+1)
        name = str(p.class_name) + ' ' + str(instance_dic[p.class_name][-1])
    else:
        instance_dic[p.class_name] = [1]
        name = str(p.class_name) + ' ' + str(int(1))
    p.instance_id = name
    print('point {} belongs to instance {}'.format(p.global_id, p.instance_id))
    if p.class_name in label_dic:
        label_dic[p.class_name].append(p)
    else:
        label_dic[p.class_name] = [p]
    return
'''
        

In [8]:
def getoutputp(p:MapPoint):
    for mid in p.mats:
        if mid in res_output:
            return res_output[mid], mid
    for mid in p.mats:
        mat_path = os.path.join(mat_dir, mats[mid])
        im = cv2.imread(mat_path)
        output = predictor(im)
        res_output[mid] = output
        return output, mid

In [9]:
def create_text_labels(classes, class_names):
    """
    Args:
        classes (list[int] or None):
        class_names (list[str] or None):

    Returns:
        list[str] or None
    """
    labels = None
    if classes is not None and class_names is not None and len(class_names) > 1:
        labels = [class_names[i] for i in classes]
    #if scores is not None:
    #    if labels is None:
    #        labels = ["{:.0f}%".format(s * 100) for s in scores]
    #    else:
    #        labels = ["{} {:.0f}%".format(l, s * 100) for l, s in zip(labels, scores)]
    return labels

In [10]:
def assign_clas(p:MapPoint, output:dict, mid:int):
    maxi = 0
    for i in range(len(output["instances"])):
        if output["instances"].pred_masks[i][int(p.mats[mid][1])][int(p.mats[mid][0])]:
            if output["instances"].scores[i] > maxi:
                p.class_id = output["instances"].pred_classes[i]
                if 'lables' not in output:
                    output['labels'] = create_text_labels(output["instances"].pred_classes, classnames)
                p.class_name = output['labels'][i]
                p.score = output["instances"].scores[i]
    if p.class_id != -1:
        pass
        #print('point {} belongs to class {}'.format(p.global_id, p.class_name))
        #print('x cor is {} and y cor is {}'.format(p.mats[mid][0], p.mats[mid][1]))

In [ ]:
#发现首先坐标没反，只是在mask那边反了一下，其他我的读取和opencv的显示都是一样的顺序

In [ ]:
#其次发现一个区域附近似乎有几个mask,laptop附近又有tv，搞得我好混乱

In [ ]:
#首先，mask是可以重叠的，所以对一个point,似乎取所属最大概率的label才比较好
#0.5这个分数似乎不能过滤很多不好的，如果改成0.6怎么样
#现在这个情况，还不能解释，为什么laptop出现了这么多个.

In [26]:
import time

mat_dir = os.path.join(os.getcwd(), 'rgb')
cnt = 0
recognize = 0 #计数，被识别的mpoint个数
#字典，用于保存一个图片识别的结果,key是id,值是output
res_output = {} #这句明显不用运行吧，要不然每次都重新算图
# 字典，保存一个类别下的mappoint，键为class_id,值是mppoint列表
label_dic = {}
# 字典，维护现存的instance,键是label名（字符串），值是int列表，
instance_dic = {}


start = time.time()

for p in points:
    output, mid = getoutputp(p)
    #print('output with length {}'.format(len(output["instances"])))
    #print('x cor is {} and y cor is {}'.format(p.mats[mid][0], p.mats[mid][1]))
    assign_clas(p, output, mid)
    if p.class_id != -1:
        print('this is mpoint {}'.format(p.global_id),'with mid {}'.format(mid), ' with {} instances are recognized'.format(len(res_output[mid]['instances'])))
        assign_inst(p)
        mat_path = os.path.join(mat_dir, mats[mid])
        im = cv2.imread(mat_path)
        #visualize(mid)
        #cv2.imshow('test',im)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #break
        recognize += 1
        #print('we got {}'.format(recognize))
    cnt += 1
    if cnt > 10000:
        break
print('{} seconds has past'.format(time.time()-start))

print('{} was recognized'.format(recognize))
store_res_output = res_output 
        
#需要观察的有mp的各项属性，label_dic,还有instance_dic

this is mpoint 41 with mid 34  with 1 instances are recognized
point 41 belongs to instance tv 1
this is mpoint 46 with mid 34  with 1 instances are recognized
point 46 belongs to instance tv 1
this is mpoint 48 with mid 34  with 1 instances are recognized
point 48 belongs to instance tv 1
this is mpoint 74 with mid 34  with 1 instances are recognized
point 74 belongs to instance tv 1
this is mpoint 75 with mid 34  with 1 instances are recognized
point 75 belongs to instance tv 1
this is mpoint 77 with mid 34  with 1 instances are recognized
point 77 belongs to instance tv 1
this is mpoint 80 with mid 34  with 1 instances are recognized
point 80 belongs to instance tv 1
this is mpoint 89 with mid 34  with 1 instances are recognized
point 89 belongs to instance tv 1
this is mpoint 94 with mid 34  with 1 instances are recognized
point 94 belongs to instance tv 1
this is mpoint 97 with mid 34  with 1 instances are recognized
point 97 belongs to instance tv 1
this is mpoint 99 with mid 34 

this is mpoint 420 with mid 73  with 8 instances are recognized
point 420 belongs to instance dining table 1
this is mpoint 421 with mid 481  with 5 instances are recognized
point 421 belongs to instance book 1
this is mpoint 424 with mid 73  with 8 instances are recognized
point 424 belongs to instance dining table 1
this is mpoint 425 with mid 73  with 8 instances are recognized
point 425 belongs to instance dining table 1
this is mpoint 426 with mid 73  with 8 instances are recognized
point 426 belongs to instance dining table 1
this is mpoint 428 with mid 73  with 8 instances are recognized
point 428 belongs to instance dining table 1
this is mpoint 429 with mid 73  with 8 instances are recognized
point 429 belongs to instance dining table 1
this is mpoint 430 with mid 73  with 8 instances are recognized
point 430 belongs to instance dining table 1
this is mpoint 431 with mid 73  with 8 instances are recognized
point 431 belongs to instance dining table 1
this is mpoint 432 with mi

this is mpoint 529 with mid 73  with 8 instances are recognized
point 529 belongs to instance dining table 1
this is mpoint 530 with mid 73  with 8 instances are recognized
point 530 belongs to instance dining table 1
this is mpoint 531 with mid 73  with 8 instances are recognized
point 531 belongs to instance dining table 1
this is mpoint 532 with mid 73  with 8 instances are recognized
point 532 belongs to instance dining table 1
this is mpoint 533 with mid 73  with 8 instances are recognized
point 533 belongs to instance dining table 1
this is mpoint 534 with mid 73  with 8 instances are recognized
point 534 belongs to instance dining table 1
this is mpoint 536 with mid 92  with 4 instances are recognized
point 536 belongs to instance dining table 1
this is mpoint 538 with mid 73  with 8 instances are recognized
point 538 belongs to instance dining table 1
this is mpoint 539 with mid 73  with 8 instances are recognized
point 539 belongs to instance dining table 1
this is mpoint 540 

point 622 belongs to instance dining table 1
this is mpoint 623 with mid 92  with 4 instances are recognized
point 623 belongs to instance dining table 1
this is mpoint 626 with mid 92  with 4 instances are recognized
point 626 belongs to instance dining table 1
this is mpoint 627 with mid 76  with 6 instances are recognized
point 627 belongs to instance tv 3
this is mpoint 628 with mid 92  with 4 instances are recognized
point 628 belongs to instance dining table 1
this is mpoint 629 with mid 73  with 8 instances are recognized
point 629 belongs to instance dining table 1
this is mpoint 630 with mid 73  with 8 instances are recognized
point 630 belongs to instance dining table 1
this is mpoint 631 with mid 73  with 8 instances are recognized
point 631 belongs to instance book 1
this is mpoint 632 with mid 92  with 4 instances are recognized
point 632 belongs to instance dining table 1
this is mpoint 633 with mid 73  with 8 instances are recognized
point 633 belongs to instance dining 

this is mpoint 730 with mid 92  with 4 instances are recognized
point 730 belongs to instance dining table 1
this is mpoint 731 with mid 92  with 4 instances are recognized
point 731 belongs to instance dining table 1
this is mpoint 733 with mid 92  with 4 instances are recognized
point 733 belongs to instance dining table 1
this is mpoint 734 with mid 92  with 4 instances are recognized
point 734 belongs to instance dining table 1
this is mpoint 735 with mid 92  with 4 instances are recognized
point 735 belongs to instance dining table 1
this is mpoint 737 with mid 92  with 4 instances are recognized
point 737 belongs to instance dining table 1
this is mpoint 738 with mid 92  with 4 instances are recognized
point 738 belongs to instance dining table 1
this is mpoint 739 with mid 92  with 4 instances are recognized
point 739 belongs to instance dining table 1
this is mpoint 740 with mid 92  with 4 instances are recognized
point 740 belongs to instance dining table 1
this is mpoint 741 

...

this is mpoint 1969 with mid 250  with 4 instances are recognized
point 1969 belongs to instance laptop 2
this is mpoint 1972 with mid 254  with 5 instances are recognized
point 1972 belongs to instance laptop 2
this is mpoint 1975 with mid 270  with 4 instances are recognized
point 1975 belongs to instance laptop 2
this is mpoint 1976 with mid 250  with 4 instances are recognized
point 1976 belongs to instance laptop 2
this is mpoint 1980 with mid 245  with 6 instances are recognized
point 1980 belongs to instance laptop 2
this is mpoint 2012 with mid 245  with 6 instances are recognized
point 2012 belongs to instance mouse 1
this is mpoint 2013 with mid 245  with 6 instances are recognized
point 2013 belongs to instance book 1
this is mpoint 2021 with mid 250  with 4 instances are recognized
point 2021 belongs to instance laptop 2
this is mpoint 2026 with mid 250  with 4 instances are recognized
point 2026 belongs to instance laptop 2
this is mpoint 2033 with mid 240  with 4 instance

this is mpoint 2230 with mid 319  with 5 instances are recognized
point 2230 belongs to instance book 1
this is mpoint 2232 with mid 319  with 5 instances are recognized
point 2232 belongs to instance book 1
this is mpoint 2233 with mid 319  with 5 instances are recognized
point 2233 belongs to instance book 1
this is mpoint 2234 with mid 319  with 5 instances are recognized
point 2234 belongs to instance book 1
this is mpoint 2235 with mid 217  with 7 instances are recognized
point 2235 belongs to instance book 1
this is mpoint 2237 with mid 217  with 7 instances are recognized
point 2237 belongs to instance book 1
this is mpoint 2241 with mid 319  with 5 instances are recognized
point 2241 belongs to instance book 1
this is mpoint 2243 with mid 319  with 5 instances are recognized
point 2243 belongs to instance book 1
this is mpoint 2246 with mid 319  with 5 instances are recognized
point 2246 belongs to instance book 1
this is mpoint 2247 with mid 319  with 5 instances are recognize

this is mpoint 2551 with mid 73  with 8 instances are recognized
point 2551 belongs to instance dining table 1
this is mpoint 2554 with mid 73  with 8 instances are recognized
point 2554 belongs to instance dining table 1
this is mpoint 2558 with mid 73  with 8 instances are recognized
point 2558 belongs to instance dining table 1
this is mpoint 2563 with mid 481  with 5 instances are recognized
point 2563 belongs to instance tv 6
this is mpoint 2572 with mid 73  with 8 instances are recognized
point 2572 belongs to instance dining table 1
this is mpoint 2576 with mid 481  with 5 instances are recognized
point 2576 belongs to instance tv 1
this is mpoint 2578 with mid 73  with 8 instances are recognized
point 2578 belongs to instance dining table 1
this is mpoint 2579 with mid 73  with 8 instances are recognized
point 2579 belongs to instance dining table 1
this is mpoint 2580 with mid 73  with 8 instances are recognized
point 2580 belongs to instance dining table 1
this is mpoint 2582

point 2862 belongs to instance dining table 1
this is mpoint 2864 with mid 528  with 4 instances are recognized
point 2864 belongs to instance remote 1
this is mpoint 2867 with mid 547  with 6 instances are recognized
point 2867 belongs to instance dining table 1
this is mpoint 2870 with mid 547  with 6 instances are recognized
point 2870 belongs to instance dining table 1
this is mpoint 2872 with mid 528  with 4 instances are recognized
point 2872 belongs to instance cup 1
this is mpoint 2874 with mid 437  with 7 instances are recognized
point 2874 belongs to instance dining table 1
this is mpoint 2876 with mid 521  with 4 instances are recognized
point 2876 belongs to instance book 4
this is mpoint 2877 with mid 99  with 5 instances are recognized
point 2877 belongs to instance dining table 1
this is mpoint 2879 with mid 521  with 4 instances are recognized
point 2879 belongs to instance book 1
this is mpoint 2880 with mid 73  with 8 instances are recognized
point 2880 belongs to ins

point 2984 belongs to instance book 4
this is mpoint 2985 with mid 535  with 3 instances are recognized
point 2985 belongs to instance book 4
this is mpoint 2986 with mid 95  with 5 instances are recognized
point 2986 belongs to instance keyboard 1
this is mpoint 2988 with mid 92  with 4 instances are recognized
point 2988 belongs to instance dining table 1
this is mpoint 2990 with mid 92  with 4 instances are recognized
point 2990 belongs to instance dining table 1
this is mpoint 2991 with mid 92  with 4 instances are recognized
point 2991 belongs to instance dining table 1
this is mpoint 2993 with mid 131  with 5 instances are recognized
point 2993 belongs to instance book 1
this is mpoint 2995 with mid 99  with 5 instances are recognized
point 2995 belongs to instance dining table 1
this is mpoint 2996 with mid 99  with 5 instances are recognized
point 2996 belongs to instance dining table 1
this is mpoint 2997 with mid 99  with 5 instances are recognized
point 2997 belongs to insta

In [18]:
#保存的th为０．６时候的结果
res_output_score_sixty

In [28]:
#保存的th为０．7时候的结果
res_output_score_seventy

In [31]:
with open("instance_saving.txt", "w") as f:
    for p in points:
        if p.class_id != -1:
            f.write(str(p.global_id))
            f.write(' ')
            f.write(str(p.instance_id))
            f.write('\n')
        

In [ ]:
res_output[34]["instances"].pred_masks[0][479][639] #图像的x坐标是６４０，x填在mask的后一位

In [16]:
points[46].instance_id

'tv 1'

In [13]:
from detectron2.utils.visualizer import Visualizer
def visualize(mid:int):
    mat_path = os.path.join(mat_dir, mats[mid])
    im = cv2.imread(mat_path)
    output = res_output[mid]
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
    v = v.draw_instance_predictions(output["instances"].to("cpu"))
    #cv2_imshow(v.get_image()[:, :, ::-1])
    cv2.imshow('test',v.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [17]:
visualize(92)

In [ ]:
len(res_output)

In [27]:
instance_dic

{'tv': [1, 2, 3, 4, 5, 6],
 'laptop': [1, 2, 3],
 'keyboard': [1, 2, 3, 4],
 'dining table': [1],
 'book': [1, 2, 3, 4, 5],
 'cup': [1],
 'mouse': [1],
 'remote': [1],
 'cell phone': [1],
 'chair': [1, 2, 3],
 'bottle': [1]}

In [ ]:
for 

In [ ]:
label_dic['laptop']

In [ ]:
points[0]

In [ ]:
for key in res_output:
    print(key)

In [ ]:
import time

mat_dir = os.path.join(os.getcwd(), 'rgb')
cnt = 0
recognize = 0 #计数，被识别的mpoint个数
#字典，用于保存一个图片识别的结果,key是id,值是output
res_output = {}
# 字典，保存一个类别下的mappoint，键为class_id,值是mppoint列表
label_dic = {}
# 字典，维护现存的instance,键是label名（字符串），值是int列表，
instance_dic = {}


start = time.time()

for point in points:
    #print(point.global_id)
    output, mid = getoutput(point)
    if k != -1:
        pass
    else:
        im = cv2.imread(mat_path)
        output = predictor(im)
        seg_output[point.mat_id] = output
        assign_clas(point, output)
        assign_inst(point, label_dic, instance_dic)
        
        
        
        

In [ ]:
a = []
for i in a:
    pass

In [ ]:
import time

#字典，用于保存一个图片识别的结果
seg_output = {}
#字典，保存一个图片id下的
mat_dir = os.path.join(os.getcwd(), 'rgb')
cnt = 0
recognize = 0
start = time.time()
for point in points:
    print(point.global_id)
    mat_path = os.path.join(mat_dir, mats[point.mat_id])
    if point.mat_id in seg_output:
        output = seg_output[point.mat_id]
        for i in range(len(output["instances"])):
            if output["instances"].pred_masks[i][int(point.y_corrdi)][int(point.x_corrdi)]:
                point.class_id = output["instances"].pred_classes[i]
                point.class_name = output['labels'][i]
                #print('point {} belongs to {}'.format(point.global_id, point.class_name))
                recognize += 1
                break #假设不存在一个像素被两个instance占用
    else:
        im = cv2.imread(mat_path)
        #cv2.imshow('test',im)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #break
        output = predictor(im)
        #print('x coor is ',len(output["instances"].pred_masks[0]))
        #print('y coor is ',len(output["instances"].pred_masks[0][0]))
        #break
        output['labels'] = _create_text_labels(output["instances"].pred_classes, output["instances"].scores, metadata.get("thing_classes", None))
        seg_output[point.mat_id] = output
        for i in range(len(output["instances"])):
            if output["instances"].pred_masks[i][int(point.y_corrdi)][int(point.x_corrdi)]:
                point.class_id = output["instances"].pred_classes[i]
                point.class_name = output['labels'][i]
                #print('point {} belongs to {}'.format(point.global_id, point.class_name))
                recognize += 1
                break
    #cv2.imshow('test',im)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()
    #print(mat_path)
    cnt += 1
    #if cnt > 50:
    #    break
print("{} points are recognized ".format(recognize))
end = time.time()
print('{} seconds has past'.format(end - start))

In [ ]:
#1589 points are recognized 
#389.0116274356842 seconds has past,宝贵的６分钟噢


In [ ]:
b[2] = 5
b[2]

In [ ]:
a = [1,2,3,3,3]
a =set(a)
a
len(a)

In [ ]:
a = set([1,2,3])
b = set([2,3])
k = a&b
k.pop()

In [ ]:
float('1.000')

In [ ]:
len(mat_set)

In [ ]:
def hasoutput(point:MapPoint, res:dict ):
    for mat_id in points.mats:
        if mat_id in res:
            return mat_id
    return -1

In [ ]:
points[2].mats

In [ ]:
b

In [ ]:
len(labels)

In [ ]:
labels

In [ ]:
import time
start = time.time()
time.sleep(5)
end = time.time()
print('{} seconds has past'.format(end - start))

In [ ]:
a = [1,2,3]
b = {1:a}
a.append(4)#1,2,3,4
a = [1,2,3,4]#b[1]还是１，２，３
b[1]
b[2] = 22

In [ ]:
for i in b:
    print(i)
    print(b[i])

In [ ]:
len(mats)

In [ ]:
float('1305031452.791720')

In [ ]:
mats[0]

In [ ]:
points[3].global_id

In [ ]:
points[5].class_name

In [ ]:
os.path.join(os.getcwd(), 'rgb')

In [ ]:
len(points)

In [ ]:
points[0].mat_id

In [ ]:
os.getcwd()

In [ ]:
mats[34]

In [ ]:
points[0].global_id

In [ ]:
with open("scmantic_saving.txt", "r") as f:
    while f.readline():
        print(f.readline())
        break

In [ ]:
mats[0]

In [ ]:
mats[-1]

In [ ]:
a = [1,2,3,4]
a[:-1]

In [ ]:
dirs[0]

In [ ]:
dirs[1]

In [ ]:
dirs[2]

In [ ]:
dirs[3]

In [ ]:
ma = MapPoint(3)
ma.id

In [ ]:
ma.mat_Id = 5
ma.mat_Id